### 前言

本篇主要介绍end-to-end的LLMops流程中的数据->SFT微调->发布->推理流程，使用的SDK版本为0.1.3。建议提前熟悉预测服务相关SDK功能作为前置知识。

In [1]:
import os
import time

# 初始化百度智能云的IAM ak, sk用于bos和千帆平台的鉴权
# 大模型平台和Bos同处于百度智能云下，所以可以使用同一个AK，SK来通过权限校验
os.environ["QIANFAN_ACCESS_KEY"] = "your_iam_ak"
os.environ["QIANFAN_SECRET_KEY"] = "your_iam_sk"

本文使用的千帆版本
```
qianfan>=0.1.3
```

## 数据上传

在进行SFT微调训练前，我们需要准备我们的训练数据；不同的训练任务需要准备不同类型的数据集，具体来说，对于LLM SFT训练任务，需要准备的是`已标注的、非排序的对话数据集`
推荐使用的数据格式为`jsonl`，即每一行文本都包含了一个json字符串，此json需要包含prompt，response两个字段，以下是一个示例，[下载](https://console.bce.baidu.com/api/qianfan/canghai/entity/static/sample-text-dialog-unsort-annotated.jsonl)：
```
[{"prompt" : "你好", "response": [["你需要什么帮助"]]}]
```
每一行表示一组数据，每组数据中的prompt和response加起来之和字符数不超过8000Token（包括中英文、数字、符号等），超出部分将被截断。

### Bos

Bos是百度智能云提供的对象存储云服务，可以高效的存取数据。本篇教程基于Bos，实现本地的数据集到千帆平台数据集的导入：

In [2]:
# 首先我们需要安装bce-python-sdk
!pip install bce-python-sdk


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from baidubce.bce_client_configuration import BceClientConfiguration
from baidubce.auth.bce_credentials import BceCredentials
from baidubce.services.bos.bos_client import BosClient

# 初始化bos配置
BosEndpoint = "bj.bcebos.com"
bucket_name = "your_bucket_name"

bos_config = BceClientConfiguration(
    credentials=BceCredentials(os.environ["QIANFAN_ACCESS_KEY"], os.environ["QIANFAN_SECRET_KEY"]),
    endpoint=BosEndpoint
)

file_name = "./data/fin_cqa_train.jsonl"
key = "/data/fin_cqa_train.jsonl"
prefix = "/data/"

bos_client = BosClient(bos_config)
bos_client.put_object_from_file(bucket_name, key, file_name)

{metadata:{date:u'Thu, 09 Nov 2023 10:50:57 GMT',content_length:u'0',connection:u'keep-alive',content_md5:u'kbo1u82WYdCFGVLAbeqXbQ==',etag:u'91ba35bbcd9661d0851952c06dea976d',server:u'BceBos',bce_content_crc_32:u'86170999',bce_debug_id:u'JUrX2nUmpvcbaRPRMsY+uS3KUFDB1YjYIbZ9aaJtEgw16FpXFpCwVQG7+iVDt2rD4dVWAh+SmNZzCEUXGOXHiQ==',bce_flow_control_type:u'-1',bce_is_transition:u'false',bce_request_id:u'b65583f2-c7fb-4fa6-ad52-c07569270120'}}

## 大模型平台鉴权介绍：

大模型平台和Bos同处于百度智能云下，所以可以使用同一个AK，SK来通过权限校验：

In [4]:
import os
# os.environ["QIANFAN_ACCESS_KEY"] = "your_iam_ak"
# os.environ["QIANFAN_SECRET_KEY"] = "your_iam_sk"

## 数据导入

在完成了以上从本地到bos的上传过程后，我们就开始着手创建数据集并导入之前上传到bos的数据

In [ ]:
from qianfan.resources import Data
from qianfan.resources.console.consts import DataSetType, DataProjectType, DataTemplateType, DataStorageType

# 创建数据集
ds = Data.create_bare_dataset(
    name="random_hi_sft_ds", 
    data_set_type=DataSetType.TextOnly,
    project_type=DataProjectType.Conversation,
    template_type=DataTemplateType.NonSortedConversation,
    storage_type=DataStorageType.PrivateBos,
    storage_id=bucket_name,
    storage_path=prefix
)

In [6]:
# 使用bos进行数据导入
from qianfan.resources.console.consts import DataSourceType

ds_id=ds["result"]["datasetId"]
import_resp = Data.create_data_import_task(
    dataset_id=ds_id,
    is_annotated=True,
    import_source=DataSourceType.PrivateBos,
    file_url="bos:/{}{}".format(bucket_name, key)
)

In [7]:
# 获取数据集详情
ds_info = Data.get_dataset_info(ds_id)

### 监听导入状态

由于数据集导入是一个耗时任务，所以我们需要等待其完成才能进行下一步的动作，这里我们通过轮询的方式简单的监听任务状态直到数据完成导入成功。

In [8]:
import time
from qianfan.resources.console.consts import DataImportStatus
while True:
    # 获取数据集详情
    ds_info = Data.get_dataset_info(ds_id)
    import_status = ds_info["result"]["versionInfo"]["importStatus"]
    if import_status == DataImportStatus.Finished.value:
        print("dataset import finish, ready to release")
        break
    print("current_import_status", import_status)
    time.sleep(10)

current_import_status 1
dataset import finish, ready to release


## 发布数据集

恭喜你到达了进行SFT训练的最后一步，我们已经完成了数据集的准备，现在需要发布数据集。
> Note：
> 发布数据集后后无法再进行数据集的处理，导入或者修改！


In [9]:
from qianfan.resources.console.consts import DataReleaseStatus

# 发布 并监听数据集发布状态
resp = Data.release_dataset(ds_id)

while True:
    # 获取数据集详情
    ds_info = Data.get_dataset_info(ds_id)
    release_status = ds_info["result"]["versionInfo"]["releaseStatus"]
    if release_status == DataReleaseStatus.Finished.value:
        print("dataset release finish, ready to train")
        break
    print("current_release_status", release_status)
    time.sleep(10)

current_release_status 1
current_release_status 1
dataset release finish, ready to train


至此，数据部分的准备已经完成！我们话不多说赶紧开始LLM的Finetune：

## Finetune

目前千帆平台支持如下 SFT 相关操作：
* 创建训练任务
* 创建任务运行
* 获取任务运行详情
* 停止任务运行

### 创建SFT任务

创建训练任务需要提供任务名称`name`和任务描述`description`，返回结果在`result`字段中，具体字段与API 文档一致。

In [10]:
from qianfan.resources import FineTune
# 创建任务
create_job_resp = FineTune.V2.create_job(
    name="random_sdk_task",
    # description="eb speed tasks",
    model="ERNIE-Speed-8K",
    train_mode="SFT",
)

# 获取任务ID
job_id = create_job_resp["result"]["jobId"]
job_id


'job-jk7fbup3znye'

### 创建任务运行

创建任务运行需要提供该次训练的详细配置，例如模型版本（`trainType`）、数据集(`trainset`)等等，且不同模型的参数配置存在差异，具体参数可以参见API 文档。

In [ ]:
# 创建任务运行，具体参数可以参见 API 文档

resp = FineTune.V2.create_task(
    job_id=job_id,
    params_scale="FullFineTuning",
    hyper_params={
        "epoch": 1,
        "learningRate": 0.00003,
        "maxSeqLen": 4096
    },
    dataset_config={
        "sourceType": "Platform",
        "versions": [{
            "versionId": ds_id
        }],
        "splitRatio": 20
    },
)

这一步会监听训练`进度`，同时也观察训练任务状态，根据训练的模型大小，方法等的不同，需要一定的时间才能进行下一步模型发布。

In [12]:
import time
task_id = resp["result"]["taskId"]
while True:
    task_status_resp = FineTune.V2.task_detail(task_id=task_id)
    task_status = task_status_resp["result"]["runStatus"]
    print("task status:", task_status)
    if task_status != 'Running':
        break
    time.sleep(60)

task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Running
task status: Done


### 发布模型

发布新模型需要指定task_id和iterationsId（job_id）；
如果是希望进行同个模型的多次迭代更新

In [13]:
from qianfan.resources import Model

sft_task_publish_resp = Model.publish(
    is_new=True,
    model_name="random_test_sdk",
    version_meta={
        "iterationId": task_id,
        "taskId": job_id,
    }
)
# 获取model_id and version
model_id =  sft_task_publish_resp["result"]["modelId"]
model_str_id = sft_task_publish_resp["result"]["modelIDStr"]
model_version_id = sft_task_publish_resp["result"]["versionId"]
model_version_str_id = sft_task_publish_resp["result"]["versionIdStr"]
print("model_id:", model_id)
print("model_str_id", model_str_id)
print("model_version", model_version_id)
print("model_version_str_id", model_version_str_id)

model_id: 17313
model_str_id am-6q6dhia9rhb8
model_version 21331
model_version_str_id amv-avbt5d6q6xb4


In [14]:
# 获取模型版本信息：
model_version_list = Model.list(model_id=model_str_id)
model_version_id: int = 0 
for m in model_version_list["result"]["modelVersionList"]:
    if m["modelIdStr"] == model_str_id and m["modelVersionIdStr"] == model_version_str_id:
        model_version_id = m["modelVersionId"]
if model_version_id == 0:
    raise ValueError("not model version")
print("model_version_id", model_version_id)

model_version_id 21331


### 监听模型版本详情状态
模型任务在训练FINISH之后，需要等待模型版本状态为READY，模型才算完全发布到模型仓库中，这一步也可以在web控制台中的我的模型/详情中看到。
这一步会进行模型发布，保存到我的模型仓库中，根据模型的大小，可能需要等待若干分钟之后才能进行下一步模型服务部署。
![my_model](img/my_model.jpg)

In [15]:
# 获取模型版本详情
# 模型版本状态有三种：Creating, Ready, Failed
while True:
    model_detail_info = Model.detail(model_version_id=model_version_str_id)
    model_version_state = model_detail_info["result"]["state"]
    print("current model_version_state:", model_version_state)
    if model_version_state != "Creating":
        break
    time.sleep(60)


current model_version_state: Ready


### 模型评估

用户可以在创建模型服务之前，先使用平台提供的模型评估功能对训练好的模型进行评估。
我们以[金融新闻摘要数据集](https://console.bce.baidu.com/qianfan/data/dataset/15067/detail)作为我们评估使用的数据集

In [16]:
eval_create_response = Model.create_evaluation_task(
    name="random_fin_test",
    version_info=[
        {
            "modelId": model_str_id,
            "modelVersionId": model_version_str_id,
        },
    ],
    dataset_id="ds-jun2ns7h96jzvwaq",
    eval_config={
        "evalMode": "rule",
        "scoreModes": [
            "similarity",
            "accuracy",
        ],
    },
    dataset_name="FinCUGE_FinNA",
).body

eval_task_id_str = eval_create_response["result"]["evalIdStr"]
print("eval_task_id", eval_task_id_str)

eval_task_id ame-8z3garj40ywr


由于评估也是一项非常耗时的任务，因此我们同样需要监听任务状态，直到评估完成。

In [17]:
# 缓冲，等待任务真正开始运行
time.sleep(5)

while True:
    eval_info = Model.get_evaluation_info(eval_task_id_str)
    eval_state = eval_info["result"]["state"]
    print("current eval_state:", eval_state)
    if eval_state != "Doing":
        break
    time.sleep(60)

current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Doing
current eval_state: Done


In [18]:
eval_result = Model.get_evaluation_result(eval_task_id_str)
print(eval_result["result"])

[{'modelName': 'test_sdk_2P36k_', 'modelVersion': '1', 'modelVersionSource': 'Train', 'evalMode': 'rule', 'evaluationName': 'fin_test_gs3Lr_', 'id': '65f96b97e169f906e3109963', 'modelVersionId': 21331, 'modelId': 17313, 'userId': 1355264, 'evaluationId': 5652, 'effectMetric': {'accuracy': 0.286, 'f1Score': 0.2552728, 'rouge_1': 0.26005724, 'rouge_2': 0.1350722, 'rouge_l': 0.24951611, 'bleu4': 0.06346849, 'avgJudgeScore': 0, 'stdJudgeScore': 0, 'medianJudgeScore': 0, 'scoreDistribution': None, 'manualAvgScore': 0, 'goodCaseProportion': 0, 'subjectiveImpression': '', 'manualScoreDistribution': None}, 'performanceMetric': {}}]


评估完成后的详细评估报告目前仅支持在[网页](https://console.bce.baidu.com/qianfan/modelcenter/model/eval/list)查看

### 创建模型服务

这一步用于创建一个在线服务，获取到service Id

In [19]:
from qianfan.resources.console.consts import DeployPoolType
from qianfan.resources import Service


g = Service.create(
    model_id = model_id, 
    model_version_id = model_version_id, 
    name="random_tests",
    uri="random_sdk",
    replicas=1, 
    pool_type=DeployPoolType.PrivateResource
)


In [20]:
svc_id = g["result"]["serviceId"]
svc_id

5729

### 部署模型服务

这一步由于需要涉及到资源的服务逻辑，所以目前需要在web上操作付费，完成付费之后即可使用模型推理服务。
![deploy_pay](img/deploy_pay.jpg)

In [86]:
# 资源付费完成后，serviceStatus会变成Deploying，查看模型服务状态, 直到serviceStatus变成部署完成，得到model_endpoint
# 这一步涉及到资源调度，需要等待5-20分钟不等
while True:
    resp = Service.get(id = svc_id)
    svc_status = resp["result"]["serviceStatus"]
    print("svc deploy status:", svc_status)
    if svc_status in ["Done", ""]:
        sft_model_endpoint=resp["result"]["uri"]
        break
    time.sleep(30)
print("sft_model_endpoint:", sft_model_endpoint)

svc deploy status: New
svc deploy status: New
svc deploy status: New
svc deploy status: New
svc deploy status: New
svc deploy status: New
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: Deploying
svc deploy status: De

### 访问SFT模型服务

在访问服务之前，首先需要配置预测服务应用的AK/SK，可以从控制台中的应用接入里获取：

![app_ak_sk](img/app_ak_sk.jpg)

In [33]:
# 预测服务的
os.environ["QIANFAN_AK"] = "your_app_ak"
os.environ["QIANFAN_SK"] = "your_app_sk"

In [39]:
import qianfan
chat_comp = qianfan.ChatCompletion(endpoint=sft_model_endpoint)
msgs = qianfan.Messages()
msgs.append(message="你好，你是谁？", role=qianfan.QfRole.User)
chat_resp = chat_comp.do(messages=msgs)
chat_resp


QfResponse(code=200, headers={'Access-Control-Allow-Headers': 'Content-Type', 'Access-Control-Allow-Origin': '*', 'Appid': '26217442', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json; charset=utf-8', 'Date': 'Thu, 26 Oct 2023 12:12:32 GMT', 'P3p': 'CP=" OTI DSP COR IVA OUR IND COM "', 'Server': 'Apache', 'Set-Cookie': 'BAIDUID=7FBBE2E6B5D83AB3C6A3B5F2ABB8E430:FG=1; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2145916555; path=/; domain=.baidu.com; version=1', 'Statement': 'AI-generated', 'Vary': 'Accept-Encoding', 'X-Aipe-Self-Def': 'eb_total_tokens:17-id:as-j0ker4w7ar', 'X-Baidu-Request-Id': '9a143024d69419cb6e8ab8bb2d751b8e1000130', 'X-Openapi-Server-Timestamp': '1698322351', 'Content-Length': '222'}, body={'id': 'as-j0ker4w7ar', 'object': 'chat.completion', 'created': 1698322352, 'result': '我是文心一言，是百度研发的。', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 5, 'completion_tokens': 12, 'total_tokens': 17}}, image=N

### 总结
至此，你已经通过SFT训练成功的微调出自己的大语言模型，SFT是一个很好的手段，用于针对于特定场景下的语料进行模型特化，以增强模型在某方面的能力，非常适合对于垂直领域内的应用。除了SFT之外，千帆平台还提供了RLHF功能，SDK也将在将来持续跟进LLMOps能力。